In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
%matplotlib inline
import tensorflow as tf
#from OpticalFlow.simplewarp import exect
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt 

import tensorflow_addons as tfa

import datetime
import sys
import numpy as np
import os
import cv2
from glob import glob
x = 0

import math
import tensorflow.compat.v1 as tf1

import argparse
tf1.disable_eager_execution()

In [2]:
#pip install flow_vis

In [3]:
import flow_vis


# warp class

In [4]:
'''
from google.colab import drive
drive.mount('/content/drive')
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n"

In [5]:
def get_pixel_value(img, x, y):
    """
    Utility function to get pixel value for coordinate
    vectors x and y from a  4D tensor image.
    Input
    -----
    - img: tensor of shape (B, H, W, C)
    - x: flattened tensor of shape (B*H*W, )
    - y: flattened tensor of shape (B*H*W, )
    Returns
    -------
    - output: tensor of shape (B, H, W, C)
    """
    shape = tf.shape(x)
    batch_size = shape[0]
    height = shape[1]
    width = shape[2]

    batch_idx = tf.range(0, batch_size)
    batch_idx = tf.reshape(batch_idx, (batch_size, 1, 1))
    b = tf.tile(batch_idx, (1, height, width))
    indices = tf.stack([b, y, x], 3)

    return tf.gather_nd(img, indices)


def tf_warp(img, flow):
    W, H = tf.cast(tf.shape(img)[2], tf.float32), tf.cast(tf.shape(img)[1], tf.float32)
    x, y = tf.meshgrid(tf.range(W), tf.range(H))
    x = tf.expand_dims(x, 0)
    x = tf.expand_dims(x, -1)

    y = tf.expand_dims(y, 0)
    y = tf.expand_dims(y, -1)

    x = tf.cast(x, tf.float32)
    y = tf.cast(y, tf.float32)
    grid = tf.concat([x, y], axis=3)

    flows = grid + flow
    max_y = tf.cast(H - 1, tf.int32)
    max_x = tf.cast(W - 1, tf.int32)
    zero = tf.zeros([], dtype=tf.int32)

    x = flows[:, :, :, 0]
    y = flows[:, :, :, 1]
    x0 = x
    y0 = y
    x0 = tf.cast(x0, tf.int32)
    x1 = x0 + 1
    y0 = tf.cast(y0, tf.int32)
    y1 = y0 + 1

    # clip to range [0, H/W] to not violate img boundaries
    x0 = tf.clip_by_value(x0, zero, max_x)
    x1 = tf.clip_by_value(x1, zero, max_x)
    y0 = tf.clip_by_value(y0, zero, max_y)
    y1 = tf.clip_by_value(y1, zero, max_y)

    # get pixel value at corner coords
    Ia = get_pixel_value(img, x0, y0)
    Ib = get_pixel_value(img, x0, y1)
    Ic = get_pixel_value(img, x1, y0)
    Id = get_pixel_value(img, x1, y1)

    # recast as float for delta calculation
    x0 = tf.cast(x0, tf.float32)
    x1 = tf.cast(x1, tf.float32)
    y0 = tf.cast(y0, tf.float32)
    y1 = tf.cast(y1, tf.float32)

    # calculate deltas
    wa = (x1 - x) * (y1 - y)
    wb = (x1 - x) * (y - y0)
    wc = (x - x0) * (y1 - y)
    wd = (x - x0) * (y - y0)

    # add dimension for addition
    wa = tf.expand_dims(wa, axis=3)
    wb = tf.expand_dims(wb, axis=3)
    wc = tf.expand_dims(wc, axis=3)
    wd = tf.expand_dims(wd, axis=3)

    # compute output
    out = tf.add_n([wa * Ia, wb * Ib, wc * Ic, wd * Id])

    return out


# Model class

In [6]:

class LiteFlowNet2():
    def __init__(self, isSintel = True):
        self.dblBackward = [0.0, 0.0, 0.0, 5.0, 2.5, 1.25, 0.625]
        self.isSintel = isSintel

    def feature_extractor(self):
        lrelu = lambda x: tf.nn.leaky_relu(x, 0.1)

        # module one
        m1 = tf.keras.Sequential()
        m1.add(tf.keras.layers.Conv2D(filters=32, kernel_size=7,  activation=lrelu, padding='SAME'))

        # module two
        m2 = tf.keras.Sequential()
        m2.add(tf.keras.layers.ZeroPadding2D(padding=(1, 1)))
        m2.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=2,  activation=lrelu, padding='valid'))
        m2.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation=lrelu, padding='SAME'))
        m2.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation=lrelu, padding='SAME'))

        # module three
        m3 = tf.keras.Sequential()
        m3.add(tf.keras.layers.ZeroPadding2D(padding=(1, 1)))
        m3.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=2, activation=lrelu, padding='valid'))
        m3.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation=lrelu,  padding='SAME'))

        # module four
        m4 = tf.keras.Sequential()
        m4.add(tf.keras.layers.ZeroPadding2D(padding=(1, 1)))
        m4.add(tf.keras.layers.Conv2D(filters=96, kernel_size=3, strides=2, activation=lrelu, padding='valid'))
        m4.add(tf.keras.layers.Conv2D(filters=96, kernel_size=3, activation=lrelu, padding='SAME'))

        # module five
        m5 = tf.keras.Sequential()
        m5.add(tf.keras.layers.ZeroPadding2D(padding=(1, 1)))
        m5.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=2, activation=lrelu, padding='valid'))

        # module six
        m6 = tf.keras.Sequential()
        m6.add(tf.keras.layers.ZeroPadding2D(padding=(1, 1)))
        m6.add(tf.keras.layers.Conv2D(filters=192, kernel_size=3, strides=2, activation=lrelu, padding='valid'))

        return [m1, m2, m3, m4, m5, m6]

    def group_upconv(self, in1, groups, name, with_bias=False):
        # keras don't have an easy way of group conv so use old way
        with tf.compat.v1.variable_scope('flownet'):
            with tf.compat.v1.variable_scope(name):
                filterc = tf.compat.v1.get_variable('filter_w', shape=[4, 4, 1, groups], dtype=tf.float32)
                shp = tf.shape(in1)
                output_shape = (shp[0], shp[1] * 2, shp[2] * 2, shp[3])
                out = tf.nn.conv2d_transpose(in1, filterc, output_shape, strides=[1, 2, 2, 1])
                if with_bias:
                    bias = tf.compat.v1.get_variable('filter_b', shape=[groups], dtype=tf.float32)
                    out = tf.nn.bias_add(out, bias)
                return out 

    def matching(self, tensor_features1, tensor_features2, tensorFlow, int_level, name):
        with tf.name_scope(name):
            lrelu = lambda x: tf.nn.leaky_relu(x, 0.1)

            def module_feat():
                return tf.keras.Sequential()
                
            def module_upcorr(x):
                return self.group_upconv(x, 49, name + '/moduleUpcorr')

            def module_upflow(x):
                return self.group_upconv(x, 2, name + '/moduleUpflow')

            def module_main(x):
                kernel_size = [0, 0, 0, 5, 5, 3, 3][int_level]                
                if kernel_size == 0:
                    raise ValueError('Should not be in level %i in Matching layer!' % int_level)
                
                with tf.name_scope('module_main'):
                    conv1 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation=lrelu, padding='SAME')(x)
                    conv2 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation=lrelu, padding='SAME')(conv1)
                    conv3 = tf.keras.layers.Conv2D(filters=96, kernel_size=3, activation=lrelu, padding='SAME')(conv2)                    
                    conv4 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation=lrelu, padding='SAME')(conv3)
                    conv5 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation=lrelu, padding='SAME')(conv4)
                    conv6 = tf.keras.layers.Conv2D(filters=2, kernel_size=kernel_size, activation=None, padding='SAME')(conv5)
                    return conv6
            
            if self.dblBackward[int_level] == 0.0:
                raise ValueError('Should not be in level %i in Matching layer!' % int_level)
            
            with tf.name_scope('module_feat'):
                m_feat = module_feat()
                tensor_features1 = m_feat(tensor_features1)
                tensor_features2 = m_feat(tensor_features2)

            if tensorFlow is not None:
                tensorFlow = module_upflow(tensorFlow)
                # warp features
                tensor_features2 = tf_warp(tensor_features2, tensorFlow * self.dblBackward[int_level])

            if int_level >= 4:
                corr = tfa.layers.optical_flow.CorrelationCost(1, 3, 1, 1, 3, 'channels_last')([tensor_features1, tensor_features2])
                corr = lrelu(corr)
            else:
                corr = tfa.layers.optical_flow.CorrelationCost(1, 6, 2, 2, 6, 'channels_last')([tensor_features1, tensor_features2])
                corr = lrelu(module_upcorr(corr))

            # hack cuz corr cost lost last dimension
            corr.set_shape([None, None, None, 49])

            return (tensorFlow if tensorFlow is not None else 0.0) + module_main(corr)

    def subpixel(self, tensor_features1, tensor_features2, tensorFlow, int_level, name='subpixel'):
        with tf.name_scope(name):
            lrelu = lambda x: tf.nn.leaky_relu(x, 0.1)

            def module_feat():
                return tf.keras.Sequential()

            def module_main(x):
                kernel_size = [0, 0, 0, 5, 5, 3, 3][int_level]
                if kernel_size == 0:
                    raise ValueError('Should not be in level %i in Subpixel layer!' % int_level)                
                
                with tf.name_scope("module_main"):
                    conv1 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation=lrelu, padding='SAME')(x)
                    conv2 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation=lrelu, padding='SAME')(conv1)
                    conv3 = tf.keras.layers.Conv2D(filters=96, kernel_size=3, activation=lrelu, padding='SAME')(conv2)                    
                    conv4 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation=lrelu, padding='SAME')(conv3)
                    conv5 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation=lrelu, padding='SAME')(conv4)
                    conv6 = tf.keras.layers.Conv2D(filters=2, kernel_size=kernel_size, activation=None, padding='SAME')(conv5)
                    if int_level > 3 or self.isSintel:
                        return conv6
                    else:
                        return [conv6, conv5]

            with tf.name_scope('module_feat'):
                mfeat = module_feat()
                tensor_features1 = mfeat(tensor_features1)
                tensor_features2 = mfeat(tensor_features2)

            if self.dblBackward[int_level] == 0:
                raise ValueError('Should not be in level %i in Subpixel layer!' % int_level) 

            tensorFlow1 = tensorFlow * self.dblBackward[int_level]
            tensor_features2 = tf_warp(tensor_features2, tensorFlow1)
            tens_flow = tf.concat([tensor_features1, tensor_features2, tensorFlow], -1)

            if int_level > 3 or self.isSintel:
                return (tensorFlow if tensorFlow is not None else 0.0) + module_main(tens_flow)
            else:
                main_output = module_main(tens_flow)
                return [(tensorFlow if tensorFlow is not None else 0.0) + main_output[0], main_output[1]]
            

    def regularization(self, tensor1, tensor2, tensor_features1, tensorFlow, int_level, name='module_regularization', intermediate_tensor=None):
        with tf.name_scope(name):
            lrelu = lambda x: tf.nn.leaky_relu(x, 0.1)
            int_unfold = [0, 0, 7, 5, 5, 3, 3][int_level]
            if int_unfold == 0:
                raise ValueError('Should not be in level %i in Regularization layer!' % int_level)            

            def module_feat(x):
                with tf.name_scope('module_feat'):
                    if int_level == 3 or int_level == 4:
                        return tf.keras.layers.Conv2D(filters=128, kernel_size=1, activation=lrelu, padding='valid')(x)
                    else:
                        return x

            moduleScale = lambda x: tf.keras.layers.Conv2D(filters=1, kernel_size=1, activation=None, padding='valid')(x)

            def module_main(x):
                if int_level > 2:
                    with tf.name_scope('module_main'):
                        conv1 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation=lrelu, padding='SAME')(x)
                        conv2 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation=lrelu, padding='SAME')(conv1)
                        conv3 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation=lrelu, padding='SAME')(conv2)
                        conv4 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation=lrelu, padding='SAME')(conv3)
                        conv5 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation=lrelu, padding='SAME')(conv4)
                        conv6 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation=lrelu, padding='SAME')(conv5)
                        return conv6
                else:
                    with tf.name_scope('moduleUpflowR'):
                        conv6 = self.group_upconv(x, 32, name + '/moduleUpflowR', True)
                        return conv6

            def module_dist(x):
                with tf.name_scope('module_dist'):
                    kernel_size = [0, 0, 7, 5, 5, 3, 3][int_level]
                    out_channels = [0, 0, 49, 25, 25, 9, 9][int_level]

                    if kernel_size == 0 or out_channels == 0:
                        raise ValueError('Should not be in level %i in Regularization layer!' % int_level)  

                    if int_level >= 5:
                        return tf.keras.layers.Conv2D(filters=out_channels, kernel_size=kernel_size, padding='SAME', activation=None,)(x)
                    else:
                        x = tf.keras.layers.Conv2D(filters=out_channels, kernel_size=(kernel_size, 1), activation=None,
                                                   padding='same')(x)
                        x = tf.keras.layers.Conv2D(filters=out_channels, kernel_size=(1, kernel_size), activation=None,
                                                   padding='same')(x)
                        return x
            
            if int_level > 2:
                if self.dblBackward[int_level] == 0:
                    raise ValueError('Should not be in level %i in Regularization layer!' % int_level) 
                
                tensor_diff = tf.sqrt(tf.reduce_sum(tf.square(tensor1 - tf_warp(tensor2, tensorFlow * self.dblBackward[int_level])),
                                      axis=3, keepdims=True))
                feat = module_feat(tensor_features1)
                intermediate_tensor = module_main(tf.concat([tensor_diff,
                                                             tensorFlow - tf.reduce_mean(tensorFlow, keepdims=True,
                                                                                         axis=[1, 2]),
                                                             feat], 3))
            else:
                intermediate_tensor = self.group_upconv(intermediate_tensor, 32, name + '/moduleUpflowR', True)
            
            tensor_dist = module_dist(intermediate_tensor)            
            tensor_dist = -tf.square(tensor_dist)
            tensor_dist = tf.exp(tensor_dist - tf.reduce_max(tensor_dist, axis=3, keepdims=True))

            tensor_div = 1. / tf.reduce_sum(tensor_dist, -1, keepdims=True)

            tensorScale = []
            moduleScaleNames = ['moduleScaleX', 'moduleScaleY']
            indices = [0, 1, 2]
            for i in range(len(moduleScaleNames)):
                with tf.name_scope(moduleScaleNames[i]):
                    tensorScale.append(moduleScale(tensor_dist *
                                                   tf.image.extract_patches(tensorFlow[..., indices[i]:indices[i+1]],
                                                                            [1, int_unfold, int_unfold, 1],
                                                                            [1, 1, 1, 1],
                                                                            [1, 1, 1, 1],
                                                                            "SAME")))
                
            if int_level != 3 or self.isSintel:
                return tf.concat([tensorScale[0] * tensor_div, tensorScale[1] * tensor_div], -1)                                                                 
            else:
                return [tf.concat([tensorScale[0] * tensor_div, tensorScale[1] * tensor_div], -1), intermediate_tensor]


    def correct_pan(self, x):
        with tf.name_scope('correct_pan'):
            lrelu = lambda x: tf.nn.leaky_relu(x, 0.1)
            conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation=lrelu, padding='SAME')(x)
            conv2 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation=lrelu, padding='SAME')(conv1)
            conv3 = tf.keras.layers.Conv2D(filters=1, kernel_size=1, activation=None, padding='valid')(conv2)

        return tf.nn.tanh(conv3)

    def module_chromas(self, x):
        with tf.name_scope('module_chromas'):
            lrelu = lambda x: tf.nn.leaky_relu(x, 0.1)
            conv1 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation=lrelu, padding='SAME')(x)
            conv2 = tf.keras.layers.Conv2D(filters=2, kernel_size=1, activation=None, padding='valid')(conv1)

        return conv2

    def __call__(self, tensor1, tensor2, scope='flownet'):
        tf.keras.backend.set_floatx('float32')
        with tf.name_scope(scope):
            tensor1_norm = tensor1 - [[[[0.411618, 0.434631, 0.454253]]]]
            tensor2_norm = tensor2 - [[[[0.410782, 0.433645, 0.452793]]]]

            m1, m2, m3, m4, m5, m6 = self.feature_extractor()

            def shared_feat_modules(x):
                with tf.name_scope('feature_extractor'):
                    t1 = m1(x)
                    t2 = m2(t1)
                    t3 = m3(t2)
                    t4 = m4(t3)
                    t5 = m5(t4)
                    t6 = m6(t5)
                return [t1, t2, t3, t4, t5, t6]

            # Extract features
            tensor_feat1 = shared_feat_modules(tensor1_norm)
            tensor_feat2 = shared_feat_modules(tensor2_norm)

            tensor1 = [tensor1_norm]
            tensor2 = [tensor2_norm]
            for i in [2, 3, 4, 5]:
                tensor1.append(tf.image.resize(tensor1[-1], tf.shape(tensor_feat1[i])[1:3]))
                tensor2.append(tf.image.resize(tensor2[-1], tf.shape(tensor_feat2[i])[1:3]))

            # Main loop
            flow = None
            lvls = [2, 3, 4, 5, 6]
            for i in [-1, -2, -3, -4]:
                flow = self.matching(tensor_feat1[i], tensor_feat2[i], flow, lvls[i], name='matching_%i' % abs(i))
                if lvls[i] == 3 and not self.isSintel:
                    [flow, intermediate_S_tensor] = self.subpixel(tensor_feat1[i], tensor_feat2[i], flow, lvls[i], name='subpixel_%i' % abs(i))                
                    [flow, intermediate_R_tensor] = self.regularization(tensor1[i], tensor2[i], tensor_feat1[i], flow, lvls[i], name='regularization_%i' % abs(i))
                else:
                    flow = self.subpixel(tensor_feat1[i], tensor_feat2[i], flow, lvls[i], name='subpixel_%i' % abs(i))
                    flow = self.regularization(tensor1[i], tensor2[i], tensor_feat1[i], flow, lvls[i], name='regularization_%i' % abs(i))                

            # Go through extra layers in Kitti model
            if not self.isSintel:
                intermediate_S_tensor = self.group_upconv(intermediate_S_tensor, 32, 'subpixel_5/moduleUpflowS', True)
                with tf.name_scope("subpixel_5/module_main"):
                    intermediate_S_tensor = tf.keras.layers.Conv2D(filters=2, kernel_size=7, activation=None, padding='SAME')(intermediate_S_tensor)
                
                flow = self.group_upconv(flow, 2, 'matching_5/moduleUpflow', False) + intermediate_S_tensor
                i = -5
                flow = self.regularization(None, None, tensor_feat1[i], flow, lvls[i], name='regularization_%i' % abs(i), intermediate_tensor=intermediate_R_tensor)

            flowr = tf.image.resize(flow, tf.shape(tensor1_norm)[1:3])

            return flowr * 20.0

# class eval

In [7]:
import math
import numpy as np
import tensorflow.compat.v1 as tf1

import argparse
tf1.disable_eager_execution()
'''
from PIL import Image
from model import LiteFlowNet2
from draw_flow import *
'''
#1000228637703

def pad_image(image):
    if len(image.shape) == 3:
        h, w, c = image.shape
    else:
        h, w = image.shape
        c = 1

    nh = int(math.ceil(h / 32.) * 32)
    nw = int(math.ceil(w / 32.) * 32)

    pad_i = np.zeros([nh, nw, c])
    pad_i[:h, :w] = image
    return pad_i

def str2bool(v):
    if isinstance(v, bool):
       return v
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')
def initsaver():
	# Create TF session
    sess = tf1.Session()
    model = LiteFlowNet2(isSintel=True)
    tens1 = tf1.placeholder(tf1.float32, shape=[None, None, None, 3])
    tens2 = tf1.placeholder(tf1.float32, shape=[None, None, None, 3])
    out = model(tens1, tens2)

    # Load model
    saver = tf1.train.Saver()
    #if args.use_Sintel:
    saver.restore(sess, "./models/LiteFlowNet2_Sintel_model")#"/content/drive/My Drive/cycleGAN/models/LiteFlowNet2_Sintel_model")
    return sess,out,tens1,tens2

	
def floweval(sess,out,inp1,inp2,tens1,tens2):
    
    w, h = inp1.shape[:2]
    inp1 = np.float32(np.expand_dims(pad_image(np.asarray(inp1)[..., ::-1]), 0)) / 255.0
    inp2 = np.float32(np.expand_dims(pad_image(np.asarray(inp2)[..., ::-1]), 0)) / 255.0

    # input in bgr format
    flow = sess.run(out, feed_dict={tens1: inp1, tens2: inp2})[0, :h, :w, :]
    
    # visualise flow with color model as image and save
    flow_color = flow_vis.flow_to_color(flow, convert_to_bgr=False)
    flow_image = Image.fromarray(flow_color)
    
    return flow_image  


In [8]:
class DataLoader():
    def __init__(self, dataset_name="default", img_res=(256, 256)):
        self.dataset_name = dataset_name
        self.img_res = img_res
        
    def load_data(self, domain, batch_size=1, is_testing=False):
        data_type = "train%s" % domain if not is_testing else "test%s" % domain
        path = glob('/datasets/%s/%s/*' % (self.dataset_name, data_type))#/content/drive/My Drive
        

        batch_images = np.random.choice(path, size=batch_size)
        #print(path)
        imgs = []
        for img_path in batch_images:
            img = self.imread(img_path)
            if not is_testing:
                #img = scipy.misc.imresize(img, self.img_res)
                img = cv2.resize(img, self.img_res, interpolation = cv2.INTER_AREA)

                if np.random.random() > 0.5:
                    img = np.fliplr(img)
            else:
                img = cv2.resize(img, self.img_res, interpolation = cv2.INTER_AREA)
            imgs.append(img)

        imgs = np.array(imgs)/127.5 - 1.

        return imgs

    def load_batch(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "val"
        path_A = glob('/datasets/%s/%sA/*' % (self.dataset_name, data_type))#/content/drive/My Drive/cycleGAN
        path_B = glob('/datasets/%s/%sB/*' % (self.dataset_name, data_type))
        #print(path_A)
        
        self.n_batches = int(min(len(path_A), len(path_B)) / batch_size)
        total_samples = self.n_batches * batch_size

        # Sample n_batches * batch_size from each path list so that model sees all
        # samples from both domains
        path_A = np.random.choice(path_A, total_samples, replace=False)
        path_B = np.random.choice(path_B, total_samples, replace=False)

        for i in range(self.n_batches-1):
            batch_A = path_A[i*batch_size:(i+1)*batch_size]
            batch_B = path_B[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img_A, img_B in zip(batch_A, batch_B):
                img_A = self.imread(img_A)
                img_B = self.imread(img_B)
                '''

                img_A = scipy.misc.imresize(img_A, self.img_res)
                img_B = scipy.misc.imresize(img_B, self.img_res)
                '''
                img_A = cv2.resize(img_A, self.img_res, interpolation = cv2.INTER_AREA)
                img_B = cv2.resize(img_B, self.img_res, interpolation = cv2.INTER_AREA)

                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B

    def imread(self, path):
        return cv2.imread(path, cv2.IMREAD_UNCHANGED).astype(np.float)
      

In [9]:
class DataLoader(DataLoader):
    global x
    def load_data_seq(self, domain, batch_size=1, is_testing=False):
        data_type = "train%s" % domain if not is_testing else "test%s" % domain
        path = glob('./datasets/%s/%s/*' % (self.dataset_name, data_type))#content/drive/My Drive/cycleGAN/
        

        batch_images = np.random.choice(path, size=batch_size)
        #print(path)
        imgs = []
        for img_path in batch_images:
            img = self.imread(img_path)
            if not is_testing:
                #img = scipy.misc.imresize(img, self.img_res)
                img = cv2.resize(img, self.img_res, interpolation = cv2.INTER_AREA)

                if np.random.random() > 0.5:
                    img = np.fliplr(img)
            else:
                img = cv2.resize(img, self.img_res, interpolation = cv2.INTER_AREA)
            imgs.append(img)

        imgs = np.array(imgs)/127.5 - 1.

        return imgs

In [10]:
class DataLoader(DataLoader):
    def load_batch_seq(self, batch_size=1, is_testing=False):
        global x
        x=batch_size + x
        print('\n',x,'\n')
        data_type = "train" if not is_testing else "val"
        path_A = glob('./datasets/%s/%sA/*' % (self.dataset_name, data_type))#/content/drive/My Drive/cycleGAN
        path_B = glob('./datasets/%s/%sB/*' % (self.dataset_name, data_type))#/content/drive/My Drive/cycleGAN
        
        self.n_batches = int(min(len(path_A), len(path_B)) / batch_size)
        total_samples = self.n_batches * batch_size

        # Sample n_batches * batch_size from each path list so that model sees all
        # samples from both domains
        #for i in range(n_batches):
        ba_A = path_A[x - batch_size : x]
        ba_B = path_B[x - batch_size : x]
            
            
        #path_A = np.random.choice(path_A, total_samples, replace=False)
        #path_B = np.random.choice(path_B, total_samples, replace=False)

        for i in range(self.n_batches-1):
            batch_A = path_A[i*batch_size:(i+1)*batch_size]
            batch_B = path_B[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img_A, img_B in zip(batch_A, batch_B):
                img_A = self.imread(img_A)
                img_B = self.imread(img_B)
                '''

                img_A = scipy.misc.imresize(img_A, self.img_res)
                img_B = scipy.misc.imresize(img_B, self.img_res)
                '''
                img_A = cv2.resize(img_A, self.img_res, interpolation = cv2.INTER_AREA)
                img_B = cv2.resize(img_B, self.img_res, interpolation = cv2.INTER_AREA)

                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B

In [11]:
class CycleGAN():
    def __init__(self):
        
        #sess and model of flownet2
        self.sess,self.out,self.tens1,self.tens2=initsaver()       
        self.img_cols=256
        self.img_rows=256
        self.channels = 3 
        #flownet2 = initflow()
        epsilon: int = 1e-5
        self.img_shape = (self.img_rows, self.img_cols,self.channels) 
        self.dataset_name = '01'
        
        self.data_loader=DataLoader(self.dataset_name)
        #self.mc=myclass()
        
        
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1) 
        self.gf = 32                                                    
        self.df = 64   
        
        """
        hyperparameters
         1, lambda_cycle — controls howstrictly the cycle-consistency loss is enforced.
         2, lambda_id — influences identity loss
         3, lambda_opt — control optical flow difference between the two network
        """
        self.lambda_cycle = 10.0  
        self.lambda_id = 0.9 * self.lambda_cycle 
        self.lambda_opt = 10.0 
        
        optimizer = Adam(0.0002, 0.5)
        
        self.d_A = self.build_discriminator('d_A')
        self.d_A.summary()
        
        self.d_B = self.build_discriminator('d_B')
        self.d_B.summary()
        
        self.d_A.compile(loss = 'mse',optimizer=optimizer,metrics=['accuracy'])
        self.d_B.compile(loss = 'mse',optimizer=optimizer,metrics=['accuracy'])
        
        self.g_AB = self.build_generator('g_AB')
        self.g_AB.summary()
        self.g_BA = self.build_generator('g_BA')
        self.g_BA.summary()
        
        img_A = tf.keras.Input(shape=self.img_shape)
        opt_A = tf.keras.Input(shape=self.img_shape)
        img_B = tf.keras.Input(shape=self.img_shape)
        opt_B = tf.keras.Input(shape=self.img_shape)
        
        print('shape of me is: ',img_A.shape[0])
        
        fake_B = self.g_AB(img_A)                                  
        fake_A = self.g_BA(img_B)
        #fake_B =myclass.flowwarp(fake_A,img_A)
        #opt_A = exect(flownet2,[img_A,img_A])
        #opt_B = img_B[0]  
    
        reconstr_A = self.g_BA(fake_B)                             
        reconstr_B = self.g_AB(fake_A)
        
        '''
        opt_A = exect(self.flownet2,[op1,op1]) 
        
        
        opt_B = exect(self.flownet2,[self.prevB,op2])
        opt_Are = exect(self.flownet2,[self.prevAre,reconstr_A])
        opt_Bre = exect(self.flownet2,[self.prevBre,reconstr_B])
        '''
        
        self.prevA = img_A
        self.prevB = img_B
        
        self.prevAre = reconstr_A
        self.prevBre = reconstr_B
        
        img_A_id = self.g_BA(img_A)                                
        img_B_id = self.g_AB(img_B)
        
        self.d_A.trainable = False                                 
        self.d_B.trainable = False
        
        valid_A = self.d_A(fake_A)                                 
        valid_B = self.d_B(fake_B)
        
        
        
        self.combined = tf.keras.Model(inputs=[img_A, img_B],
                              outputs=[valid_A, valid_B,
                                       reconstr_A, reconstr_B,    
                                       img_A_id, img_B_id]) 
        self.combined.compile(loss=['mse', 'mse','mae', 'mae','mae', 'mae'],
                              loss_weights=[1, 1, self.lambda_cycle, self.lambda_cycle, self.lambda_id, 
                                            self.lambda_id],
                              optimizer=optimizer)
        self.combined.summary()    

In [12]:
class CycleGAN(CycleGAN):
    @staticmethod  
    def conv2d(layer_input, num_Filter, f_size=4, Strides=1,Ins_Norm=True,Padding='same'):
        d = layers.Conv2D(num_Filter,kernel_size=f_size,strides=Strides, padding=Padding)(layer_input)
        if Ins_Norm:
            d = tfa.layers.InstanceNormalization(axis=3, center=True, epsilon=1e-5)(d)
            #d = layers.BatchNormalization()(d)
        d = layers.LeakyReLU(alpha=0.2)(d)
        return d
    
    @staticmethod
    def resNET(layer_input, skip_input, filters, f_size=4,Strides=1, dropout_rate=0):
        
        r = layers.Conv2D(filters, kernel_size=f_size, strides=Strides, padding='same', activation='relu')(layer_input)
        #r = layers.BatchNormalization()(r)
        r = tfa.layers.InstanceNormalization(axis=3, center=True, epsilon=1e-5)(r)
        r = layers.Concatenate()([r,skip_input])
        return r
  
    @staticmethod
    def deconv2d(layer_input, num_Filter, f_size=4, Strides=1,Ins_Norm=True,Padding='same'):
        d = layers.Conv2DTranspose(num_Filter,kernel_size=f_size,strides=Strides, padding=Padding)(layer_input)
        if Ins_Norm:
            #d = layers.BatchNormalization()(d)
            d = tfa.layers.InstanceNormalization(axis=3, center=True, epsilon=1e-5)(d)
        d = layers.LeakyReLU(alpha=0.2)(d)
        return d
  

In [13]:
class CycleGAN(CycleGAN):
    def build_generator(self,name):
        d0 = tf.keras.Input(self.img_shape)  
        ''' ConvNET '''
        d1 = self.conv2d(d0, 64, 7)              
        d2 = self.conv2d(d1, 128, 3, 2, Ins_Norm=True)          
        d3 = self.conv2d(d2, 256, 3, 2, Ins_Norm=True)
        
        ''' Residual Block '''
        r1 = self.conv2d(d3, 256, 3, 1)
        r2 = self.conv2d(r1, 256, 3, 1)
        r3 = self.conv2d(r2, 256, 3, 1)
        r4 = self.conv2d(r3, 256, 3, 1)
        r5 = self.conv2d(r4, 256, 3, 1)
        
        r6 = self.resNET(r5, r4, 256, 3, 1)
        r7 = self.resNET(r6, r3, 256, 3, 1)
        r8 = self.resNET(r7, r2, 256, 3, 1)
        r9 = self.resNET(r8, r1, 256, 3, 1)
        
        u1 = self.deconv2d(r9, 128, 3, 2, True)
        u2 = self.deconv2d(u1, 64, 3, 2, True)
                        
        output_img = layers.Conv2D(self.channels, kernel_size=7, strides=(1,1), padding='same',activation='tanh')(u2) 
        x=tf.keras.Model(d0, output_img,name=name)
        #x.Summary()
        return x
    
    def build_discriminator(self,n):
        #conv2d(layer_input, num_Filter, f_size=4, Strides=1,Ins_Norm=True,Padding='same'):
        img=tf.keras.Input(self.img_shape)
        d1 = self.conv2d(img, 64, 4, 2)
        d2 = self.conv2d(d1, 128, 4, 2)
        d3 = self.conv2d(d2, 256, 4, 2)
        d4 = self.conv2d(d3, 512, 4, 2)  
        d5 = self.conv2d(d4, 1, 4, 1)  
       
        x=tf.keras.Model(img,d5,name=n)
        
        return x

In [14]:
class CycleGAN(CycleGAN):
    def sample_images(self, epoch, batch_i):
        r, c = 2, 3
        imgs_A = self.data_loader.load_data(domain="A", batch_size=1, is_testing=True)
        imgs_B = self.data_loader.load_data(domain="B", batch_size=1, is_testing=True)    
        # Translate images to the other domain
        fake_B = self.g_AB.predict(imgs_A)
        #fake_A = self.g_BA.predict(imgs_B)
        '''
        cheeck here
        '''
        # Translate back to original domain
        reconstr_A = self.g_BA.predict(fake_B)
        reconstr_B = self.g_AB.predict(fake_A)
        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        titles = ['Original', 'Translated', 'Reconstructed']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                #print(gen_imgs[cnt].shape)
                b,g,r = cv2.split(gen_imgs[cnt])       # get b,g,r
                im_rgb = cv2.merge([r,g,b])     # switch it to rgb
                
                axs[i,j].imshow((im_rgb * 255).astype(np.uint8))
                axs[i, j].set_title(titles[j])
                axs[i,j].axis('on')
                cnt += 1
        fig.savefig("images/%s/%d_%d.png" % (self.dataset_name, epoch, batch_i))
        
        plt.show()
        

In [15]:
class CycleGAN(CycleGAN):
    def train(self, epochs, batch_size=1, sample_interval=50):
        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)
        for epoch in range(epochs):
            
            for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch_seq(batch_size)):
               
                # ----------------------
                #  Train Discriminators
                # ----------------------
                # Translate images to opposite domain
                
                fake_B = self.g_AB.predict(imgs_A)
                fake_A = self.g_BA.predict(imgs_B)
                print(fake_A.shape,imgs_A.shape)
                
                ###################
                fake_A = floweval(self.sess,self.out,fake_A[0],fake_A[0],self.tens1,self.tens2)
                ###################
                
                #imagewarp()
                
                # Train the discriminators (original images = real / translated = Fake)
                dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
                dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
                dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)
                dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
                dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
                dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)
                # Total discriminator loss
                d_loss = 0.5 * np.add(dA_loss, dB_loss)
                
                # ------------------
                #  Train Generators
                # ------------------
                #print(d_loss,dA_loss,dB_loss)
                # Train the generators
                print(self.combined.metrics_names)
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B],
                                                      [valid, valid,
                                                       imgs_A, imgs_B,
                                                       imgs_A, imgs_B])
                
                print(g_loss)
                
                #print('g_loss: ',g_loss)
                # If at save interval => plot the generated image samples
                
                if batch_i % sample_interval * 50 == 0:
                    self.sample_images(epoch, batch_i)

In [16]:
cycle_gan = CycleGAN()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Restoring parameters from ./models/LiteFlowNet2_Sintel_model
Model: "d_A"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d_98 (Conv2D)           (None, 128, 128, 64)      3136      
_________________________________________________________________
instance_normalization (Inst (None, 128, 128, 64)      128       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_99 (Conv2D)           (None, 64, 64, 128)       131200    
_________________________________________________________________
instance_normalization_1 (In (None, 6

Model: "g_BA"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d_121 (Conv2D)             (None, 256, 256, 64) 9472        input_4[0][0]                    
__________________________________________________________________________________________________
instance_normalization_24 (Inst (None, 256, 256, 64) 128         conv2d_121[0][0]                 
__________________________________________________________________________________________________
leaky_re_lu_20 (LeakyReLU)      (None, 256, 256, 64) 0           instance_normalization_24[0][0]  
_______________________________________________________________________________________________

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
g_BA (Model)                    (None, 256, 256, 3)  8137731     input_7[0][0]                    
                                                                 g_AB[1][0]                       
                                                                 input_5[0][0]                    
______________________________________________________________________________________________

In [ ]:
cycle_gan.train(epochs=10, batch_size=1, sample_interval=2)


 1 

(1, 256, 256, 3) (1, 256, 256, 3)
